<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/LH/lh_ac_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [0]:
#uploaded = files.upload()
'''Load data'''
#ac_data = pd.read_csv(io.BytesIO(uploaded['lh_ac.csv']))
ac_data = pd.read_csv('lh_ac.csv')

In [0]:
window_size=60
dataX_raw = create_dataset(ac_data['meter_power'].as_matrix(), window_size)

In [0]:
dataX_raw.shape

(426081, 60)

In [0]:

Y_raw = ac_data['device_power'].values[window_size-1:,].reshape([-1,1])
Y_raw.shape
categorized_Y = np.ones(Y_raw.shape)*3
categorized_Y[[np.where(Y_raw==0)[0]],:]=0
categorized_Y[[np.where((Y_raw>0)&(Y_raw<=100))[0]],:]=1
categorized_Y[[np.where((Y_raw>100)&(Y_raw<=1000))[0]],:]=2
categorized_Y[np.where((Y_raw>1000))[0],:]=3

encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
Y_1hot = encoder.fit_transform(categorized_Y)

X_train, X_test, y_train, y_test = train_test_split(dataX_raw, Y_1hot, test_size=0.1, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
X_test

array([[0.10129715, 0.10127345, 0.10124976, ..., 0.09994655, 0.09992286,
        0.09989916],
       [0.03805395, 0.03805398, 0.03805402, ..., 0.03805583, 0.03805586,
        0.03805589],
       [0.03726404, 0.03726408, 0.03726411, ..., 0.03726592, 0.03726595,
        0.03726599],
       ...,
       [0.00898379, 0.00899061, 0.00899742, ..., 0.00937232, 0.00937913,
        0.00938595],
       [0.01987111, 0.02121375, 0.02255639, ..., 0.01315789, 0.02631579,
        0.        ],
       [0.01151316, 0.0120614 , 0.01260965, ..., 0.00935189, 0.00913441,
        0.00891692]])

In [0]:
df = pd.Series(categorized_Y.flatten())
df.value_counts()

0.0    242622
1.0    153865
3.0     20356
2.0      9238
dtype: int64

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.0001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=512
unit_num=128

In [0]:
'''RNN Model Definition'''
tf.reset_default_graph()
''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1],name='x')
tf_y = tf.placeholder(tf.int32, [None, 4],name='y')

lstm_cell =tf.keras.layers.LSTMCell(units=unit_num,name='lstm_cell')
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
    lstm_cell,                   # cell you have chosen
    tf_x,                      # input
    initial_state=None,         # the initial hidden state
    dtype=tf.float32,           # must given if set initial_state = None
    time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)
l1 = tf.layers.dense(outputs[:, -1, :],512,activation=tf.nn.leaky_relu,name='l1')
l2 = tf.layers.dense(l1,1024,activation=tf.nn.leaky_relu,name='l2')
l3 = tf.layers.dense(l2,512,activation=tf.nn.leaky_relu,name='l3')
l4 = tf.layers.dense(l3,256,activation=tf.nn.leaky_relu,name='l4')
l5 = tf.layers.dense(l4,128,activation=tf.nn.leaky_relu,name='l5')
l6 = tf.layers.dense(l5,84,activation=tf.nn.leaky_relu,name='l6')
l7 = tf.layers.dense(l6,64,activation=tf.nn.leaky_relu,name='l7')
l8 = tf.layers.dense(l7,48,activation=tf.nn.leaky_relu,name='l8')
l9 = tf.layers.dense(l8,32,activation=tf.nn.leaky_relu,name='l9')
l10 = tf.layers.dense(l9,24,activation=tf.nn.leaky_relu,name='l10')
l11 = tf.layers.dense(l10,16,activation=tf.nn.leaky_relu,name='l11')
l12 = tf.layers.dense(l11,8,activation=tf.nn.leaky_relu,name='l12')
pred = tf.layers.dense(l12,4,activation=tf.nn.relu,name='pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:8000,1:8000,2:8000,3:8000})

sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net_ac/save_net_rnn.ckpt')

for i in range(0,10):
  X_train, y_train = tl.fit_resample(X_train, y_train)
  whole_train = np.concatenate([X_train,y_train],axis=1)
  for j in range(0,100):
    print('Loop'+str(i)+',###iteration: '+str(j)+'###')
    batch_index = np.random.choice(32000,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-4].reshape([batch_size,window_size,1])
    batch_y = batch_train[:,-4:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
    acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y:y_test})
    print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
    print('Test Acc=%.2f'% acc_test)
    
    pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
    y_lables_argmax = tf.argmax(tf_y,axis=1)  
    y_pred_argmax = tf.argmax(pre,axis=1)
    confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
    #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
    print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
    if(j//99==0):
      save_path = saver.save(sess, "my_net_ac_new1/save_net_rnn.ckpt")
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from my_net_ac/save_net_rnn.ckpt
Loop0,###iteration: 0###
train loss= 0.6189, Acc=0.76
Test Acc=0.84
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
[[ 91   5   8  12]
 [ 23 118   2   1]
 [ 37  15  56  12]
 [  2   0   4 126]]
Loop0,###iteration: 1###
train loss= 0.5039, Acc=0.81
Test Acc=0.84
[[108   4   4  10]
 [ 16 100   1   0]
 [ 17  23  75  15]
 [  1   0   4 134]]
Loop0,###iteration: 2###
train loss= 0.5505, Acc=0.78
Test Acc=0.84
[[109   6   7   6]
 [ 24 107   1   0]
 [ 34  19  57  13]
 [  2   0   0 127]]
Loop0,###iteration: 3###
train loss= 0.5910, Acc=0.80
Test Acc=0.84
[[118   4   6   5]
 [ 24 115   4   0]
 [ 26  14  62  13]
 [  1   0   6 114]]
Loop0,###iteration: 4###
train loss= 0.5390, Acc=0.82
Test Acc=0.84
[[123   4  10   6]
 [ 17 101   5   0]
 [ 23  11  70  10]
 [  4   0   4 124]]
Loop0,###iteration: 5#

In [0]:
!mkdir my_net_ac

mkdir: cannot create directory ‘my_net_ac’: File exists


In [0]:
mv checkpoint save_net_rnn.ckpt.data-00000-of-00001 save_net_rnn.ckpt.index save_net_rnn.ckpt.meta my_net_ac

In [0]:
mv my_net_ac my_net

In [0]:
rm -r my_net_ac_new

In [0]:
import pickle

file_name='scaler'
fileObject = open(file_name,'wb')
pickle.dump(scaler,fileObject)
fileObject.close()

In [0]:
#Generate Predictions for the raw data by batch
dataX = dataX_raw
batch_size = 64
batch_num = len(dataX)//batch_size
nn_dataX = None
print(batch_num)
#restore rnn session
sess = tf.Session()
saver.restore(sess, 'my_net_ac_new1/save_net_rnn.ckpt')
for i in range(0,batch_num+1):
  print(i)
  if(i!=batch_num):
      batch_X = dataX[i*batch_size:(i+1)*batch_size,]
      batch_X = scaler.transform(batch_X).reshape([-1,window_size,1])
      batch_pred = sess.run(pred,feed_dict={tf_x: batch_X})
      if nn_dataX is None:
        nn_dataX = batch_pred
      else:
        nn_dataX = np.vstack([nn_dataX,batch_pred])
      
  else: 
      batch_X = dataX[i*batch_size:]
      batch_X = scaler.transform(batch_X).reshape([-1,window_size,1])
      batch_pred = sess.run(pred,feed_dict={tf_x: batch_X})
      nn_dataX = np.vstack([nn_dataX,batch_pred])

6657
INFO:tensorflow:Restoring parameters from my_net_ac_new1/save_net_rnn.ckpt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
25

In [0]:
result_df = pd.DataFrame(data=nn_dataX)
result_df.to_csv('nn_data.csv',index=False)